In [1]:
import requests,pandas,geopandas,shapely,os

In [2]:
def converter(results_dir,label,url,relevant_fields):
    
    print('Processing {} ...'.format(label))
    
    # 1. read url
    response=requests.get(url)
    data=response.json()
    
    # 2. create dictionary
    dictionary={}
    for field in relevant_fields:
        dictionary[field]=[]
        
    print('\t Detected {} locations.'.format(len(data['items'])))

    for item in data['items']:    

        item_id=item['id']
        titol=item['titol']
        phones=','.join([element for element in item['telefons'] if len(element) > 3])

        try:
            utmx=item['utm']['utmx']
            utmy=item['utm']['utmy']

            dictionary['id'].append(item_id)
            dictionary['titol'].append(titol)
            dictionary['utx'].append(utmx)
            dictionary['uty'].append(utmy)
            dictionary['phone'].append(phones)

        except:
            print('WARNING: dropping {} because utm not found.'.format(titol))
    
    # 3. create pandas dataframe and write csv
    df=pandas.DataFrame.from_dict(dictionary)
    df.to_csv('{}{}.locations.txt'.format(results_dir,label),sep='\t')
    
    # 4. create geopandas dataframe
    geometry=[shapely.geometry.Point(xy) for xy in zip(df.utx,df.uty)]
    crs={'init':'epsg:25831'}
    gdf=geopandas.GeoDataFrame(df,crs=crs,geometry=geometry)
    
    # 5. create shapefile
    gdf.to_file('{}{}.locations.shp'.format(results_dir,label))
    
    return None

# 0. User defined variables

In [3]:
url_file='//Users/adrianlopezgarciadelomana/gd/help/toni/data/url_serveisoberts.txt'
results_dir='/Users/adrianlopezgarciadelomana/gd/help/toni/results/all/'

In [4]:
relevant_fields=['id','titol','utx','uty','phone']

In [5]:
tags={
    'Ajuntaments':'ajuntaments',
    'Confraries':'conf_pesc',
    'atenci%C3%B3%20ciutadana':'of_aten_ciut',
    'agr%C3%A0ria':'eca',
    'capacitaci%C3%B3%20nauticopesquera':'ecanautica',
    'Hotels':'hotels_entitats',
    'Biblioteques':'biblioteca',
    'Cultura&adv':'cultura',
    'Educaci%C3%B3':'educacio',
    'Depend':'cme',
    'Bombers':'bombers',
    'Policia%20de%20Tr%C3%A0nsit%':'transit_mossos',
    'Policies%20Locals':'polic_local',
    'Centres%20sanitaris':'centres_sanitaris',
    'deixalleries':'deixalleries',
}

# 1. Read data

In [6]:
# not working because:
# DriverError: '/vsimem/b90a75d3a81f4494ac7992a7be08428d' not recognized as a supported file format.

# geopandas.read_file(url)

In [7]:
urls=[]
with open(url_file,'r') as f:
    for line in f:
        v=line.split()
        urls.append(v[0])
print('URLs detected: {}'.format(len(urls)))

URLs detected: 15


In [8]:
# check that each tag is uniquely URL
for tag in tags:
    found=sum(tag in s for s in urls)
    if found != 1:
        raise ValueError('Tag found more than in one URL.')

# 2. Iterate URLs

In [9]:
for url in urls:
    
    # select the label for the output file
    for tag in tags:
        if tag in url:
            label=tags[tag]
            
    # convert json to shapefile
    converter(results_dir,label,url,relevant_fields)

Processing ajuntaments ...
	 Detected 500 locations.
Processing conf_pesc ...
	 Detected 30 locations.
Processing of_aten_ciut ...
	 Detected 7 locations.
Processing eca ...
	 Detected 13 locations.
Processing ecanautica ...
	 Detected 1 locations.
Processing hotels_entitats ...
	 Detected 12 locations.
Processing biblioteca ...
	 Detected 500 locations.
Processing cultura ...
	 Detected 500 locations.
Processing educacio ...
	 Detected 500 locations.
Processing cme ...
	 Detected 103 locations.
Processing bombers ...
	 Detected 146 locations.
Processing transit_mossos ...
	 Detected 8 locations.
Processing polic_local ...
	 Detected 215 locations.
Processing centres_sanitaris ...
	 Detected 500 locations.
Processing deixalleries ...
	 Detected 484 locations.
